In [89]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [90]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain, SequentialChain

llm_model = "gpt-3.5-turbo"
llm = ChatOpenAI(temperature=0.9, model=llm_model)

In [97]:
url_contact = os.environ["URL_CONTACT"]
url_newsletter = os.environ["URL_NEWSLETTER"]
system_msg = "You are a world class interior designer that specializes in the Transitional Design style.  You have a knack for explaining concepts related to interior design in a way that is accessible and engaging.  You avoid using the passive voice in any sentence.  Your sentences should be no more than 20 words long."
outline_human_msg = """Please write a blog post outline with 5 sections in addition to the introduction and conclusion in the first person about "{input}" 
"""
write_section_msg="""
I would like you to craft each section with enough detail for three paragraphs each.
I would like each section, including the introduction and the conclusion should be preceded with a "####" delimiter.  
"""
write_introduction_msg="""
Include an introduction that indicates that this article is part of my series of interior design blog posts called the "{series_title}" series.
This should be 3 paragraphs long.
"""
write_conclusion_msg="""

"""
write_call_to_action_msg="""
write a call to action that will appearat the end of the full blog post to contact me for a free consultation or to sign up for my newsletter. 
The link to my newsletter is {url_newsletter}
The "Contact Me" link should have a URL of {url_contact}
"""
write_seo_msg="""
Examine this blog post and make sure that it is SEO friendly. 
Please list a good seo keyphrase and a meta description at the end. 
"""


In [98]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessage,
    HumanMessagePromptTemplate,
    PromptTemplate,
)
system_message_prompt = SystemMessage(
    content=(system_msg)
)
human_message_prompt = HumanMessagePromptTemplate(
        prompt=PromptTemplate(
            template=outline_human_msg,
            input_variables=["input"],
            # input_variables=["input", "series_title", "url_contact", "url_newsletter"],
        )
    )

In [99]:
main_template = ChatPromptTemplate.from_messages([
    system_message_prompt,
    human_message_prompt
])
draft_chain = LLMChain(llm=llm, prompt=main_template, output_key="first_draft")

In [100]:
input="How to place picture frames on a bookshelf for maximum effect."

In [101]:
draft_chain.run({"input": input})

'Introduction:\n- Briefly explain the importance of picture frames in interior design and the impact they can have on a bookshelf.\n\nSection 1: Choosing the Right Frames\n- Discuss the importance of selecting frames that complement the overall style and color scheme of the room.\n- Offer tips on mixing different frame sizes and shapes to create visual interest.\n\nSection 2: Grouping and Arranging\n- Explain how to group frames together based on size, shape, or theme.\n- Provide guidance on arranging frames in an asymmetrical or balanced manner for a visually pleasing look.\n\nSection 3: Layering and Stacking\n- Describe how to layer frames by placing smaller ones in front of larger ones.\n- Offer tips on stacking frames vertically or horizontally to add depth and dimension to the bookshelf.\n\nSection 4: Incorporating Other Decorative Items\n- Suggest adding decorative items such as small sculptures or vases to enhance the overall display.\n- Explain how to position these items strat

In [103]:
outline='Introduction:\n- Briefly explain the importance of picture frames in interior design and the impact they can have on a bookshelf.\n\nSection 1: Choosing the Right Frames\n- Discuss the importance of selecting frames that complement the overall style and color scheme of the room.\n- Offer tips on mixing different frame sizes and shapes to create visual interest.\n\nSection 2: Grouping and Arranging\n- Explain how to group frames together based on size, shape, or theme.\n- Provide guidance on arranging frames in an asymmetrical or balanced manner for a visually pleasing look.\n\nSection 3: Layering and Stacking\n- Describe how to layer frames by placing smaller ones in front of larger ones.\n- Offer tips on stacking frames vertically or horizontally to add depth and dimension to the bookshelf.\n\nSection 4: Incorporating Other Decorative Items\n- Suggest adding decorative items such as small sculptures or vases to enhance the overall display.\n- Explain how to position these items strategically to complement the picture frames.\n\nSection 5: Showcasing Personal Memories\n- Encourage incorporating personal photographs or sentimental items into the display.\n- Explain how to intersperse these items with other decorative frames to create a personalized touch.\n\nConclusion:\n- Recap the key points discussed throughout the blog post, emphasizing the importance of creativity and personalization.\n- Encourage readers to experiment with different arrangements and styles to find what works best for their bookshelf.'

In [ ]:
style_instructions = "As always, you should keep your sentences to a maximum of 20 words and never write a sentence in the passive voice."
section_1_prompt=ChatPromptTemplate.from_template(
    """Please write 5 paragraphs on the topic of Section 1 in the outline provided below.
    
    ### OUTLINE
    {outline}
    
    {style_instructions}"""
)

ChatPromptTemplate.from_messages([
    system_message_prompt,
    human_message_prompt
])
section_1_chain = LLMChain(llm=llm, prompt=section_1_prompt)
# section_1_chain = LLMChain(llm=llm, prompt=section_1_prompt, output_key="first_draft")
section_1_chain.run({"outline": outline, "style_instructions": style_instructions})

# section_1_chain = LLMChain(llm=llm, prompt=section_1_prompt, output_key="expanded_section_1")

In [71]:
section_1_prompt=ChatPromptTemplate.from_template(
    "Could you rewrite Section 1 of this blog post and provide more detail?"
    "\n\n{first_draft}"
)

section_1_chain = LLMChain(llm=llm, prompt=section_1_prompt, output_key="expanded_section_1")

In [72]:
section_2_prompt=ChatPromptTemplate.from_template(
    "Could you rewrite Section 2 of this blog post and provide more detail?"
    "\n\n{first_draft}"
)

section_2_chain = LLMChain(llm=llm, prompt=section_2_prompt, output_key="expanded_section_2")

In [73]:
second_draft_prompt=ChatPromptTemplate.from_template(
    "Now, please assemble these re"
    "\n\n FIRST DRAFT: {first_draft}"
    "\n\n REWRITTEN SECTION 1: {expanded_section_1}"
)

second_draft_chain = LLMChain(llm=llm, prompt=second_draft_prompt, output_key="second_draft")

In [79]:
overall_chain = SequentialChain(
    chains = [draft_chain, section_1_chain, section_2_chain],
    input_variables = ["input", "series_title", "url_contact", "url_newsletter"],
    output_variables = ["first_draft", "expanded_section_1", "expanded_section_2"],
    verbose=True
)

In [80]:
output = overall_chain({
    'input': "How to place picture frames on a bookshelf for maximum effect.",
    'series_title': "Do It Yourself",
    'url_newsletter': "foo",
    'url_contact': "bar",
    })



> Entering new SequentialChain chain...

> Finished chain.


In [87]:
sec1 = output["expanded_section_1"]
sec2 = output["expanded_section_2"]
first_draft = output["first_draft"]

In [88]:

print(sec2)

#### Section 2: The Power of Layering

Now that you've mastered the importance of alignment, let's explore the power of layering when it comes to placing picture frames on a bookshelf. Bookshelves are not just for holding books, they're an opportunity to showcase your personal taste and style. So why not make the most of it? By incorporating layers, you can add depth and visual interest to your bookshelf display.

To begin, select a variety of frames in different sizes, shapes, and colors. This diverse selection will create a dynamic and visually captivating arrangement. Don't shy away from including frames that are unique and eye-catching.

When arranging your frames, consider layering them in front of books or other objects on the shelves. By placing a smaller frame in front of a larger one, you can create a sense of depth and dimension. This technique draws the eye in and adds a three-dimensional quality to the display.

For the more adventurous decorator, try leaning some frames ag